In [2]:
import torch
import dgl

In [25]:
graph_name = "ogbn-arxiv"
TARGET_DIR = "/home/ubuntu/data/" + graph_name
cache_size_in_GB = "1"
with open(TARGET_DIR + "/meta.txt",'r') as fp:
    lines = fp.readlines()
meta = {}
for line in lines:
    k, v = line.split("=")
    meta[k] = int(v)

with open(TARGET_DIR + "/partition_offsets.txt", 'r') as fp:
    lines = fp.readlines()
partition_offsets = lines[0].split(",")    
partition_offsets = [int(i) for i in partition_offsets]
print(meta)
print(partition_offsets)

{'num_nodes': 169343, 'num_edges': 1166243, 'feature_dim': 128, 'csum_features': 444011, 'csum_labels': 3501834, 'csum_train': 7744621894, 'csum_test': 2503057778, 'csum_offsets': 99706604203, 'csum_edges': 98781606055, 'num_classes': 40}
[0, 43580, 87187, 129587, 169343]


In [58]:
cache_size = "2MB"
num_nodes = meta['num_nodes']
if cache_size[-2:] == "GB":
    size_in_bytes = (1024 ** 3) * int(cache_size[:-2])
if cache_size[-2:] == "MB":
    size_in_bytes = (1024 ** 2) * int(cache_size[:-2])

percentage_of_nodes_to_cache = size_in_bytes/(meta['num_nodes'] * meta['feature_dim'] *  32)
percentage_of_nodes_to_cache = min(1, percentage_of_nodes_to_cache)
nodes_to_cache = int(percentage_of_nodes_to_cache * num_nodes)

In [41]:
fsize = meta['feature_dim']
import numpy as np
num_partitions = 4
ordered_partition_nodes = []
for partition in range(num_partitions):
    ordered_partition_nodes.append(\
            (torch.arange(partition_offsets[partition],partition_offsets[partition + 1])))    
out_degree = torch.from_numpy(np.fromfile(TARGET_DIR + "out_degrees.bin",dtype = np.int32))


In [59]:
# Create order book foook
orderbook = []
for curr_partition in range(num_partitions):
    global_nodes_not_in_partition = []
    local_ordering = []
    nodes_to_cache_for_partition = nodes_to_cache
    for i in range(num_partitions):
        if (i == curr_partition):
            global_nodes_in_partition = ordered_partition_nodes[i]
        else:
            global_nodes_not_in_partition.append(ordered_partition_nodes[i])
        local_ordering.append(partition_offsets[i + 1])
    num_nodes_from_self =  min(nodes_to_cache, global_nodes_in_partition.shape[0])
    nodes_to_cache_for_partition -= num_nodes_from_self
    local_ordering[curr_partition] = partition_offsets[curr_partition] + \
                                                    num_nodes_from_self
    if(nodes_to_cache_for_partition == 0):
        orderbook.append(local_ordering)
        continue
    
    global_nodes_not_partition = torch.cat(global_nodes_not_in_partition)
    not_in_partition_degree = out_degree[global_nodes_not_partition]
    values, indices = torch.sort(not_in_partition_degree)
    global_nodes_not_in_partition_cached = indices[:nodes_to_cache_for_partition]
    for i in range(num_partitions):
        if i == curr_partition:
            continue
        selected =  (global_nodes_not_in_partition_cached >= partition_offsets[i] ) \
            & (global_nodes_not_in_partition_cached < partition_offsets[i + 1])
        local_ordering[i] = partition_offsets[i] + global_nodes_not_in_partition_cached[selected].shape[0]   
    orderbook.append(local_ordering)
    
print(orderbook)


[[512, 87187, 129587, 169343], [43580, 44092, 129587, 169343], [43580, 87187, 87699, 169343], [43580, 87187, 129587, 130099]]


In [60]:
with open(f"{TARGET_DIR}/order_book_{cache_size}.txt", 'w') as fp:
    for p_offsets in orderbook:
        str_offsets = [str(p) for p in p_offsets]
        str_offsets = ",".join(str_offsets)
        fp.write(f"{str_offsets}\n")